In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv


**Text Preprocessing**

In [2]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [3]:
df=pd.read_csv("/kaggle/input/dataset-tachygraphy/Tachygraphy_dataset_main.csv")
df.shape

(4958, 3)

In [4]:
df.head()

,Text,Meaning,Sentiment
0,Last session of the day http://twitpic.com/67ezh,Last session of the day http://twitpic.com/67ezh,neutral
1,Shanghai is also really exciting (precisely --...,Shanghai is also really exciting (precisely --...,positive
2,submit the report ASAP!,submit the report as soon as possilbe,negative
3,happy bday!,Happy Birthday!,positive
4,The OGs - I like it!!,The original gangsters - i like it!,positive


In [5]:
df=df.drop('Meaning',axis=1)
df = df.rename(columns={'Text': 'tweet'})
df = df.rename(columns={'Sentiment': 'class'})
df['class'] = df['class'].str.lower()
sentiment_counts = df['class'].value_counts()
sentiment_counts

class
neutral     1743
positive    1643
negative    1572
Name: count, dtype: int64

In [6]:
df.isnull().sum()

tweet    1
class    0
dtype: int64

In [7]:
df=df.fillna('')

In [8]:
import re
from bs4 import BeautifulSoup

def text_cleaning(text):
    text = str(text)
    soup = BeautifulSoup(text, "html.parser")    ###removing html tages
    text = re.sub(r'\[[^]]*\]', '', soup.get_text())      ##removing text within square brackets
    pattern = r"[^a-zA-Z0-9\s,']"                  # Removing unwanted characters
    text = re.sub(pattern, '', text)
    url_pattern = r'http\S+|www\S+'            ###removing urls
    text = re.sub(url_pattern, '', text)
    return text

In [9]:
df['tweet']=df['tweet'].apply(text_cleaning).tolist()

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
import re

In [11]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [12]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [13]:
df['tweet'].dtype

dtype('O')

In [14]:
df['class'].dtype

dtype('O')

In [15]:
df.tail(10)

,tweet,class
4948,time flies by when you're facetiming the bae,positive
4949,U dont know how I feel about this,negative
4950,come to my job I'll give you a baconator lol,positive
4951,Rihanna just made Maya Angelou her avi you're...,negative
4952,im the lil bitch in the theaters tryna hold ba...,negative
4953,make a pet face wtf wrong with me tonight haha,negative
4954,I dnt care anymore Boyz ain't worth d drama,negative
4955,No relationship is perfect tho me bae goo from...,negative
4956,Over here tryna get my nail polishes and shit lol,negative
4957,No one was loved d way i luv U,positive


In [16]:
df = df[df['class'] != "neutral"]  ###droping neutral columns
df['tweet'] = df['tweet'].apply(lambda x: x.lower())

print(df[ df['class'] == 'positive'].size)
print(df[ df['class'] == 'negative'].size)

for idx,row in df.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(df['tweet'].values)
X = tokenizer.texts_to_sequences(df['tweet'].values)
X = pad_sequences(X)

3286
3144


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Embedding
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import RMSprop
from keras_tuner import RandomSearch

In [18]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-08-21 20:33:13--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-08-21 20:33:13--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-08-21 20:33:13--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: 'glove.6B.zip'

glov

In [19]:
embedding_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs


In [20]:
# max_features = 5000  
# embed_dim = 128 
# lstm_out = 64  
# batch_size = 32
# epochs = 20
# learning_rate = 0.001

In [21]:
max_features = 10000  
embed_dim = 128 
lstm_out = 64  
batch_size = 64
epochs = 20
initial_learning_rate = 0.0005

In [22]:
embedding_dim = 100
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in tokenizer.word_index.items():
    if i < max_features:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

In [23]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense


# Model
model = Sequential()
model.add(Embedding(max_features, embedding_dim, embeddings_initializer=Constant(embedding_matrix),
                    input_length=X.shape[1], trainable=True))
model.add(SpatialDropout1D(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(LSTM(64, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))  # Assuming 2 classes for classification

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())

In [24]:
tf.random.set_seed(42)

In [25]:
optimizer = Adam(learning_rate=initial_learning_rate)

In [26]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.001), metrics=['accuracy'])

In [27]:
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d               │ ?                      │   0 (unbuilt) │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [28]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, min_lr=0.00001, verbose=1)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss', mode='min')


In [29]:
Y = pd.get_dummies(df['class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(2572, 29) (2572, 2)
(643, 29) (643, 2)


In [30]:
validation_size = 500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]

In [31]:
# batch_size = 32
# model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

# Training
history = model.fit(X_train, Y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, Y_test),
                    verbose=2,
                    callbacks=[early_stopping, reduce_lr, model_checkpoint])

Epoch 1/20
41/41 - 13s - 310ms/step - accuracy: 0.5393 - loss: 1.1534 - val_accuracy: 0.4825 - val_loss: 1.0562 - learning_rate: 0.0010
Epoch 2/20
41/41 - 2s - 47ms/step - accuracy: 0.6236 - loss: 0.9255 - val_accuracy: 0.4825 - val_loss: 1.2326 - learning_rate: 0.0010
Epoch 3/20
41/41 - 5s - 115ms/step - accuracy: 0.6738 - loss: 0.7835 - val_accuracy: 0.5734 - val_loss: 1.0250 - learning_rate: 0.0010
Epoch 4/20
41/41 - 5s - 120ms/step - accuracy: 0.6979 - loss: 0.6920 - val_accuracy: 0.6014 - val_loss: 0.8846 - learning_rate: 0.0010
Epoch 5/20
41/41 - 5s - 114ms/step - accuracy: 0.7135 - loss: 0.6331 - val_accuracy: 0.6853 - val_loss: 0.6963 - learning_rate: 0.0010
Epoch 6/20
41/41 - 5s - 115ms/step - accuracy: 0.7286 - loss: 0.5905 - val_accuracy: 0.7063 - val_loss: 0.6851 - learning_rate: 0.0010
Epoch 7/20
41/41 - 2s - 43ms/step - accuracy: 0.7500 - loss: 0.5433 - val_accuracy: 0.6643 - val_loss: 0.7881 - learning_rate: 0.0010
Epoch 8/20

Epoch 8: ReduceLROnPlateau reducing learning

In [32]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, Y_test)
print(f"Test accuracy: {test_acc:.4f}")

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.7834 - loss: 0.5229
Test accuracy: 0.7902


In [33]:
score,acc = model.evaluate(X_test, Y_test,verbose = 2, batch_size = batch_size)
print("loss_score: %.2f" % (score))
print("accuracy: %.2f" % (acc))

3/3 - 0s - 20ms/step - accuracy: 0.7902 - loss: 0.5263
loss_score: 0.53
accuracy: 0.79


In [34]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):
    
    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
   
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1
       
    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

1/1 - 0s - 348ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 27ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 29ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 42ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 24ms/step
1/1 - 0s - 26ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 23ms/step
1/1 - 0s - 22ms/step
1/1 - 0s - 25ms/step
1/1 - 0s - 2

In [35]:
twt = ['I been told you that yo folks dont care about you . Im starving and my dad deadass didnt make me a plate']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    1   79  477    7   18  559 1000
    39  275   55    7   10 1913    6    5  458  165  118   13    4 1562]]
1/1 - 0s - 339ms/step
negative


In [36]:
model.save('lstm_model.h5')